A friend posed this question:

> Miguel Cabrera passed up Babe Ruth on the career hits list last night, meaning that there is now no player in history with more career hits, more career home runs, and a higher batting average than Miguel Cabrera. I wonder 
how many players that can be said about. Ruth, Aaron, Bonds, Cobb, Rose…but who else?

Here I set out to answer it, first with a brute-force approach, and then with a more efficient approach.

In [1]:
import pandas as pd
import numpy as np
from functools import reduce

import boxball_loader as bbl
import utils

In [2]:
bat = pd.read_parquet('../data/baseballdatabank/batting.parquet')

careers = bat.groupby('player_id')[['ab', 'h', 'hr']].sum()
careers['ba'] = (careers['h']/careers['ab']).fillna(0)

careers

,ab,h,hr,ba
player_id,,,,
aardsda01,4,0,0,0.000000
aaronha01,12364,3771,755,0.304998
aaronto01,944,216,13,0.228814
aasedo01,5,0,0,0.000000
abadan01,21,2,0,0.095238
...,...,...,...,...
zupofr01,18,3,0,0.166667
zuvelpa01,491,109,2,0.221996
zuverge01,142,21,0,0.147887


In [3]:
# Find players who match this player's career totals in all of the categories
def find_matches(row, df):
    categories = ['h', 'hr', 'ba']
    cat_matches = [df[cat]>=row[cat] for cat in categories]
    matches_all = reduce((lambda x, y: x & y), cat_matches)
    
    return df[matches_all]
    

In [4]:
# This should return two guys tied
find_matches({'ba': 1, 'h': 2, 'hr': 1}, careers)

,ab,h,hr,ba
player_id,,,,
oconnfr01,2,2,1,1.0
yanes01,2,2,1,1.0


In [5]:
# The brute force method iterates over every single player,
# Finding how many guys match his totals, and filtering
# for those with only one match (themselves)
def get_unmatchables_brute_force(careers):

    careers_matching = careers.apply(lambda row: len(find_matches(row, careers)), axis=1)
    unmatchables = careers[careers_matching==1]
    return unmatchables.sort_values(by='h')

In [6]:
unmatchables = get_unmatchables_brute_force(careers)
unmatchables, len(unmatchables)

(              ab     h   hr        ba
 player_id                            
 paciojo01      3     3    0  1.000000
 jansera01      5     4    0  0.800000
 brittza01      8     5    1  0.625000
 silvelu01     11     6    0  0.545455
 durhado01     14     7    2  0.500000
 dappecl01     17     8    1  0.470588
 martido02     22     9    2  0.409091
 heathsl01     25    10    2  0.400000
 pattepa01     35    14    1  0.400000
 willigl01     40    17    0  0.425000
 rhombke01     47    18    1  0.382979
 goodahe01     45    19    0  0.422222
 forstte01     78    31    0  0.397436
 hayeske01     85    32    5  0.376471
 willite01   7706  2654  521  0.344407
 ruthba01    8398  2873  714  0.342105
 simmoal01   8759  2927  307  0.334171
 hornsro01   8173  2930  301  0.358497
 bondsba01   9847  2935  762  0.298060
 gwynnto01   9288  3141  135  0.338178
 musiast01  10972  3630  475  0.330842
 aaronha01  12364  3771  755  0.304998
 cobbty01   11436  4189  117  0.366299
 rosepe01   14053  4256  

In [7]:
# This is an iterative approach, which starts with the observation
# that anybody who leads in one category must be unmatchable.
# And that anybody who can't match that player in at least one
# category has no chance to be unmatchable, because they are
# completely dominated already.  So iterate by taking the current
# leader, and pruning everybody with no chance at matching, until
# there's nobody left.  Then clean up for ties.
def get_unmatchables_iterative(careers):
    categories = ['h', 'hr', 'ba']

    def run_one_iteration(unmatchables, df):
        leader = df.head(1)

        # Keep any player who beats our leader in at least one category
        # Discard the rest
        cat_matches = [df[cat]>leader[cat][0] for cat in categories]
        beats_any = reduce((lambda x, y: x | y), cat_matches)
        remain = df[beats_any]
        print (leader.iloc[0].name, len(remain)) # to see the iterative process
        return unmatchables.append(leader), remain
    
    df = careers.sort_values(by=categories, ascending=False)
    unmatchables = careers.head(0)
    while len(df) > 0:
        unmatchables, df = run_one_iteration(unmatchables, df)
    
    # there might be some ties, so run the original definition again to filter out the real answers
    careers_matching = unmatchables.apply(lambda row: len(find_matches(row, careers)), axis=1)
    return unmatchables[careers_matching==1]

In [8]:
unmatchables = get_unmatchables_iterative(careers)
unmatchables, len(unmatchables)

rosepe01 1460
cobbty01 926
aaronha01 475
musiast01 435
gwynnto01 433
bondsba01 432
hornsro01 429
simmoal01 428
ruthba01 423
willite01 422
hayeske01 394
forstte01 381
goodahe01 334
rhombke01 333
willigl01 330
pattepa01 326
oconnda01 306
heathsl01 304
martido02 303
dappecl01 298
durhado01 129
silvelu01 128
brittza01 119
jansera01 95
paciojo01 2
oconnfr01 0


(              ab     h   hr        ba
 player_id                            
 rosepe01   14053  4256  160  0.302853
 cobbty01   11436  4189  117  0.366299
 aaronha01  12364  3771  755  0.304998
 musiast01  10972  3630  475  0.330842
 gwynnto01   9288  3141  135  0.338178
 bondsba01   9847  2935  762  0.298060
 hornsro01   8173  2930  301  0.358497
 simmoal01   8759  2927  307  0.334171
 ruthba01    8398  2873  714  0.342105
 willite01   7706  2654  521  0.344407
 hayeske01     85    32    5  0.376471
 forstte01     78    31    0  0.397436
 goodahe01     45    19    0  0.422222
 rhombke01     47    18    1  0.382979
 willigl01     40    17    0  0.425000
 pattepa01     35    14    1  0.400000
 heathsl01     25    10    2  0.400000
 martido02     22     9    2  0.409091
 dappecl01     17     8    1  0.470588
 durhado01     14     7    2  0.500000
 silvelu01     11     6    0  0.545455
 brittza01      8     5    1  0.625000
 jansera01      5     4    0  0.800000
 paciojo01      3     3  

In [9]:
unmatchables['name'] = utils.get_player_names_df(unmatchables, idx_fld='player_id')
print(unmatchables[['name', 'ba', 'hr', 'h', 'ab']].to_string())

                       name        ba   hr     h     ab
player_id                                              
rosepe01          Pete Rose  0.302853  160  4256  14053
cobbty01            Ty Cobb  0.366299  117  4189  11436
aaronha01        Hank Aaron  0.304998  755  3771  12364
musiast01       Stan Musial  0.330842  475  3630  10972
gwynnto01        Tony Gwynn  0.338178  135  3141   9288
bondsba01       Barry Bonds  0.298060  762  2935   9847
hornsro01    Rogers Hornsby  0.358497  301  2930   8173
simmoal01        Al Simmons  0.334171  307  2927   8759
ruthba01          Babe Ruth  0.342105  714  2873   8398
willite01      Ted Williams  0.344407  521  2654   7706
hayeske01    Ke'Bryan Hayes  0.376471    5    32     85
forstte01     Terry Forster  0.397436    0    31     78
goodahe01      Herb Goodall  0.422222    0    19     45
rhombke01    Kevin Rhomberg  0.382979    1    18     47
willigl01    Glenn Williams  0.425000    0    17     40
pattepa01     Pat Patterson  0.400000    1    14